In [ ]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [2]:
df = pd.read_csv("C:\\Users\ACER\\anaconda3\\envs\\py310\\ipynb\\data\\ClaMP_Integrated-5184.csv", sep=',')

In [3]:
df.drop(['packer_type'],axis=1,inplace=True)

In [4]:
from sklearn.utils import shuffle
df = shuffle(df)

In [5]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [6]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [7]:
df.drop(['index'],axis=1,inplace=True)

In [8]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.697975,0.000000,67128,5.861573,0,0
1,144,3,4,65535,184,200,5,1,1,1,...,1,0,5,0,6.513043,0.000000,320000,7.928679,0,1
2,144,3,4,65535,184,216,5,1,0,1,...,1,1,4,0,6.265370,4.419514,800256,5.761428,1,0
3,80,2,4,65535,184,124,4,1,1,1,...,1,0,4,0,5.327202,0.062379,17920,5.298230,0,1
4,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.585629,1.661954,127464,6.120273,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,1,6.344432,1.800035,221184,5.942623,1,0
5206,144,3,4,65535,184,256,4,1,0,1,...,1,0,4,0,7.790837,0.000000,885264,7.763619,0,1
5207,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,3.839223,0.000000,125952,5.311190,0,1
5208,144,3,4,65535,184,192,3,1,1,1,...,1,2,1,1,0.000000,0.000000,160768,7.958257,0,1


In [9]:
df_5percent=df[0:5210]

In [10]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.697975,0.000000,67128,5.861573,0,0
1,144,3,4,65535,184,200,5,1,1,1,...,1,0,5,0,6.513043,0.000000,320000,7.928679,0,1
2,144,3,4,65535,184,216,5,1,0,1,...,1,1,4,0,6.265370,4.419514,800256,5.761428,1,0
3,80,2,4,65535,184,124,4,1,1,1,...,1,0,4,0,5.327202,0.062379,17920,5.298230,0,1
4,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.585629,1.661954,127464,6.120273,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,1,6.344432,1.800035,221184,5.942623,1,0
5206,144,3,4,65535,184,256,4,1,0,1,...,1,0,4,0,7.790837,0.000000,885264,7.763619,0,1
5207,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,3.839223,0.000000,125952,5.311190,0,1
5208,144,3,4,65535,184,192,3,1,1,1,...,1,2,1,1,0.000000,0.000000,160768,7.958257,0,1


In [11]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [12]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [13]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
1      144.0   3.0        4.0     65535.0  184.0     200.0               5.0   
2      144.0   3.0        4.0     65535.0  184.0     216.0               5.0   
3       80.0   2.0        4.0     65535.0  184.0     124.0               4.0   
4      144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
5205   144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
5206   144.0   3.0        4.0     65535.0  184.0     256.0               4.0   
5207   144.0   3.0        4.0     65535.0  184.0     248.0               5.0   
5208   144.0   3.0        4.0     65535.0  184.0     192.0               3.0   
5209   144.0   3.0        4.0     65535.0  184.0     200.0               4.0   

      C

In [14]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.0,...,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5.210000e+03,5210.000000,5210.000000,5210.000000
mean,152.658733,10.633589,8.903263,65137.618234,202.530902,222.833013,4.662956,0.990403,0.362764,1.0,...,0.999040,1.360845,3.302111,0.156430,4.939586,2.523549,7.875718e+05,6.364756,0.544146,0.522457
std,616.499070,392.745064,251.685103,5005.606042,968.625098,48.591497,1.922882,0.097502,0.480844,0.0,...,0.030967,1.623972,1.146956,0.363297,2.521787,2.685909,5.293440e+06,1.119881,0.498095,0.499543
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.886006,0.000000,6.144000e+04,5.695785,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.148211,1.555973,1.218560e+05,6.390618,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.507072,4.685836,3.055480e+05,7.314531,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [15]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [16]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [17]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [18]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      51.25
0.0      48.75
Name: proportion, dtype: float64

In [19]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
0.0      50.0
1.0      50.0
Name: proportion, dtype: float64

In [20]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [21]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00543743 -0.0061543  -0.01197806]
  [ 0.02141658 -0.00094621 -0.00928828]]

 [[-0.0088523  -0.00412188 -0.00482619]
  [ 0.00164165  0.00233095  0.00117995]]

 [[ 0.01462378  0.01538715 -0.02439106]
  [ 0.00603441 -0.00251044 -0.00163867]]

 [[-0.0147633   0.01486981 -0.00024455]
  [ 0.00355551  0.00417011  0.00832462]]

 [[-0.00293399 -0.00029839  0.00095126]
  [ 0.00664654 -0.00140218 -0.00033193]]] 0.0


In [22]:
circuit(weights_init, data[0])

tensor(-0.00255736, requires_grad=True)

In [23]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [24]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [25]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [26]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [27]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(1.74785929, requires_grad=False), tensor(0.00970829, requires_grad=False)], Y =  1
X = [tensor(-0.18354445, requires_grad=False), tensor(-0.07982928, requires_grad=False)], Y =  1
X = [tensor(-0.32973904, requires_grad=False), tensor(0.14291528, requires_grad=False)], Y = -1
X = [tensor(-0.18406701, requires_grad=False), tensor(-0.07781849, requires_grad=False)], Y = -1
X = [tensor(-0.18724501, requires_grad=False), tensor(-0.06502611, requires_grad=False)], Y =  1


In [28]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [ ]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1087107 | Accuracy: 0.5125000 
Iter:     2 | Cost: 1.6141169 | Accuracy: 0.5125000 
New best
Iter:     3 | Cost: 1.5222753 | Accuracy: 0.5187500 
Iter:     4 | Cost: 1.1466404 | Accuracy: 0.5187500 
Iter:     5 | Cost: 0.9682483 | Accuracy: 0.5000000 
New best
Iter:     6 | Cost: 0.9703004 | Accuracy: 0.5375000 
Iter:     7 | Cost: 0.9886561 | Accuracy: 0.5000000 
Iter:     8 | Cost: 0.9516105 | Accuracy: 0.5375000 
New best
Iter:     9 | Cost: 0.9197583 | Accuracy: 0.7062500 
Iter:    10 | Cost: 0.9325083 | Accuracy: 0.6125000 
Iter:    11 | Cost: 0.9158894 | Accuracy: 0.7062500 
Iter:    12 | Cost: 0.9056837 | Accuracy: 0.7062500 
New best
Iter:    13 | Cost: 0.8977753 | Accuracy: 0.7187500 
Iter:    14 | Cost: 0.9040133 | Accuracy: 0.7062500 
Iter:    15 | Cost: 0.9054524 | Accuracy: 0.5187500 
Iter:    16 | Cost: 1.0188179 | Accuracy: 0.5250000 
Iter:    17 | Cost: 1.1043914 | Accuracy: 0.5250000 
Iter:    18 | Cost: 1.1012730 | Accuracy: 0.5187500 
I

In [ ]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [ ]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.9447533512360566, Accuracy: 52.0%


In [ ]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,-1.0
1,1.0,-1.0
2,1.0,-1.0
3,1.0,-1.0
4,1.0,-1.0
5,1.0,1.0
6,1.0,1.0
7,1.0,-1.0
8,1.0,1.0
9,1.0,-1.0


In [ ]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         1
         1.0       0.95      0.54      0.69        39

    accuracy                           0.53        40
   macro avg       0.48      0.27      0.34        40
weighted avg       0.93      0.53      0.67        40

0.9545454545454546
0.5384615384615384
0.6885245901639344
0.2692307692307692
